In [155]:
#Extra

#1 - Export environment variables

In [1]:
#Main

import sys
import subprocess
import time
from datetime import datetime

"""A cada minuto lê a tabela dos pares e, para todos aqueles cujo campo "get_data" estiver marcado como
"on" (e cujo campo "status" não seja "building"), chamará um subprocesso para completar o dado, até o tempo
presente. Deve escrever na mesma tabela dos pares o PID do subprocesso evocado para cada par. Caso a tabela
de dados para aquele i-ésimo par não exista, será criada.
"""

keys = {'open_time': 'timestamp', 'open': 'numeric', 'high': 'numeric', 'low': 'numeric', 'close': 'numeric', 
        'volume': 'numeric', 'quote_asset_volume': 'numeric', 'number_of_trades': 'integer', 
        'taker_buy_base_asset_volume': 'numeric', 'taker_buy_quote_asset_volume': 'numeric'}

default_log = 'print'

round_count = 0
while True:
    
    msg = '''ENTRADA (ciclo: ''' + str(round_count) + ''')
=================================================================================================
FROM.: Main
AT...: ''' + str(datetime.utcnow()) + '''

'''
        
    log_handler(msg,default_log)

    #Busca pela tabela de pares
    pairs = read_table('binance_pairs',mute = 'yes')
    
    if (len(pairs) > 0): #Enconta a tabela dos pares
        
        n_pairs_on = 0
        for pair in pairs:
    
            if (pair['get_data'] == 'ON'):
        
                n_pairs_on+=1
        
        if(n_pairs_on > 0): 
            
            max_requests = int(1100/n_pairs_on)
        
            for pair in pairs:
                
                pid = 23

                if (pair['get_data'] == 'ON'):
                    
                    candle_interval = '1m' #Futuramente, pode ser passada por um parâmetro.

                    table_name = 'binance_klines_' + str(pair['name']) + '_' + candle_interval
                    
                    #Tenta buscar a tabela de candles para este par, retornando a amostra gravada mais recentemente
                    last_saved_candle = read_table(table_name, field_key = 'open_time', 
                                                   sort_type = 'DESC', limit = '1', mute = 'yes')

                    if (len(last_saved_candle) > 0): #Encontra a respectiva tabela de candles
                        
                        if (pair['status'] == 'full'):
                            
                            start_time = str(int(datetime.timestamp(last_saved_candle[0]['open_time'])*1000) + 30000)
                            
                            #data_completer(pair['name'], candle_interval, start_time, max_requests)
                            
                            pid = subprocess.Popen([sys.executable, "data_completer.py", str(pair['name']), 
                                                    candle_interval, start_time, max_requests)
                            
                            update_table_job_status = update_table('binance_pairs', 'name', str(pair['name']), 
                                                                   last_change_by_PID, pid)
                                                    
                            #AQUI SERÁ TRATADO O CENÁRIO EM QUE O STATUS ESTÁ "BUILDING", PORÉM O PID CAIU
                    
                    else: #NÃO Encontra a respectiva tabela de candles
                        
                        start_time = '1241893500000'
                        
                        create_table_job_satus = create_table(table_name,keys,pk='open_time')
                        
                        data_completer(pair['name'], candle_interval, start_time, max_requests)
                    
                    msg = '''
{ciclo: ''' + str(round_count) + ''' ,par: ''' + str(pair['name']) + ''' ,PID: ''' + str(pid) + ''' ,max_requests: ''' + str(max_requests) + '''} 
'''
        
                    log_handler(msg,default_log)

            sleep_time = 60 - int(datetime.utcnow().second)

        else: #n_pairs_on = 0
            
            sleep_time = 300
            
            print('nenhum par marcado com get_data "ON".')
    
    else: #Não encontra a tabela de PARES.

        print('tabela de pares não encontrada')

        sleep_time = 300
    
    msg = '''
FROM.: Main
AT...: ''' + str(datetime.utcnow()) + '''
=================================================================================================
SAÍDA (ciclo: ''' + str(round_count) + '''). Dormindo ''' + str(sleep_time) + ''' segundos

'''
        
    log_handler(msg,default_log)
    
    round_count+=1
    
    time.sleep(sleep_time)

SyntaxError: invalid syntax (<ipython-input-1-e28c94c3f458>, line 71)

In [ ]:
                    msg = '''
FROM: Main
''' + str(datetime.utcnow()) + '''

{'ciclo':' ''' + str(round_count) + ''' ', 'par':' ''' + str(pair['name']) + ''' ', 'PID':' ''' + str(PID) + ''' ', 'max_requests':' ''' + str(max_requests) + '''} 

'''

In [1]:
#data_completer.py

#Import
import os
import sys
import time
#from modules import general_functions.loghandler as loghandler

#FAZER A DOCSTRING
#FORMATAR AS MENSAGENS DE LOGS + LOGHANDLER

def data_completer(pair, interval, start_time, max_requests):
    
    #interval = str(os.environ.get('candle_interval'))
    
    #pair = str(sys.argv[1])
    
    #start_time = str(sys.argv[2])
    
    #max_requests = int(sys.argv[3])
    
    #table_name = 'binance_klines_' + pair + '_' + interval
    
    keys = {'open_time': 'timestamp', 'open': 'numeric', 'high': 'numeric', 'low': 'numeric', 'close': 'numeric',
        'volume': 'numeric', 'quote_asset_volume': 'numeric', 'number_of_trades': 'integer',
       'taker_buy_base_asset_volume': 'numeric', 'taker_buy_quote_asset_volume': 'numeric'}
    
    max_attempts = 10
    
    #Atualiza o status do par em operação para 'building'
    update_table_job_status = update_table('binance_pairs', 'name', pair, 'status', 'building')
    #É preciso tratar a possível exceção update_table_job_status = 'fail'
    
    print('Iniciando complemento de dados para o par ',pair)
    print(' ')
    print('Atualizando para building: ', update_table_job_status)
    print(' ')
    
    total_num_requests = 0
    round_counter = 1
    
    while True:
        
        print('ciclo ', round_counter)
        klines = []
        num_requests_each_round = 0
        
        klines, num_requests_each_round = binance_klines_request_handler(pair, interval, start_time, max_attempts)
        
        total_num_requests = total_num_requests + num_requests_each_round
        
        if (len(klines) > 0):
            
            print('klines vieram!')
            
            if (len(klines) < 500): #Chegou na kline mais recente
                
                klines = klines[:(len(klines)-1)] #Apaga o último resgistro (candle não fechado)
                
                
                
                if (len(klines) == 0): 
                    
                    print(' ')
                    print('************************')
                    print('KLINE NULA')
                    print('************************')
                    print(' ')
                    
                    break
                
                
            
            most_recent_open_time = klines[len(klines)-1][0] #open_time do último candle
            
            klines = binance_klines_to_postgres_klines(klines) #Parse das klines
            
            save_in_table_job_status = save_in_table(table_name, keys, klines)
            
            start_time = str(most_recent_open_time + 30000) #Um passo de 30s (30000 milissegundos)
            
            print ('Leva de ',len(klines),' salvas no banco. Total de requests até agora: ',total_num_requests)
            print(' ')
            
            if (len(klines) < 500):
                
                data_completer_job_status = 'done'
                
                print('kline mais recente')
                

                break

            if (len(klines) > 500): #Alguma anomalia.

                #print('ATENÇÃO, MAIS DE 500 AMOSTRAS')
                print('ANOMALIA')
                
                break
                
        else: #ERRO DE COMUNICAÇÃO COM A BINANCE descansa e trata log
            
            print('ERRO DE COMUNICAÇÃO COM A BINANCE')
        
        if(total_num_requests > int(max_requests)):

            print('número máximo de requests atingido, dormindo 1 min')
            print(' ')
            total_num_requests = 0 #zera o contador de requests
            time.sleep(60)
            
        round_counter+=1

    #Atualiza o status do par em operação para 'full'
    update_table_job_status = update_table('binance_pairs', 'name', pair, 'status', 'full')
    #É preciso tratar a possível exceção update_table_job_status = 'fail'
    
    print(' ')
    print('Atualizando para full: ', update_table_job_status)
    print(' ')
    print('FIM')

In [2]:
#Db functions


#Import

import psycopg2
import psycopg2.extras
import os
from datetime import datetime
#from general_functions import log_handler
#from modules import general_functions.loghandler as loghandler

#Environment
'''
POSTGRES_DB = os.environ.get('POSTGRES_DB')
POSTGRES_USER = os.environ.get('POSTGRES_USER')
POSTGRES_PASSWORD = os.environ.get('POSTGRES_PASSWORD')
DB_HOST = os.environ.get('DB_HOST')
'''
#APAGAR####################################
DB_HOST='localhost'
POSTGRES_DB='pg_dev' 
POSTGRES_PASSWORD='06Fj@%r7KTXm5+eWn2'
POSTGRES_USER='openbot'
###########################################

#Functions

def create_table(table_name,keys,**kwargs):
    
    """Cria a tabela <table_name>, com as colunas <keys[i]>. Opcionalmente pode-se definir uma das chaves como 
    chave primária, ao invés da chave default atribuida pelo postgres.
    
    
    Keyword arguments:
    =================
    
    table_name  -- Nome da tabela a ser criada
    keys        -- Chaves (nomes das colunas)


    **kwargs:
    ========
    
    pk   -- Chave que será atribuida como primária 
    mute -- Suprime o log se 'yes'
    """
    
    pk = kwargs.get('pk')
    
    mute = kwargs.get('mute')
    
    connection = '' #Inicialmente definida como string
    
    #NÃO DEVERIA ESTAR AQUI
    #records = [] #Inicialmente definida como uma lista vazia
    
    create_table_job_status = 'fail' 
    
    default_log = 'print'
    
    if(mute):
        
        if(mute == 'yes'): default_log = 'mute'
    
    #Tenta conectar ao banco
    try:

        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)

        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
    
    #Não conecta ao banco
    except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI

        msg = '''
FROM.: create_table
AT...: ''' + str(datetime.utcnow()) + '''

Não foi possível conectar ao banco devido ao erro: 
''' + str(error) + '''.

'''
        
        log_handler(msg,default_log)

    finally:

        if (type(connection) != str): #Conexão bem sucedida com o banco de dados    
    
            #Tenta criar a tabela
            try:
                
                sql_create_query = sql_command(table_name, keys, 'create')
                
                if (pk):
                    
                    sql_create_query = sql_command(table_name, keys, 'create', pk=pk)
                
                pointer.execute(sql_create_query)
                
                connection.commit()
                
                create_table_job_status = 'done'

            #Erro aocriar tabela
            except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI
                
                #Desfaz ação do ponteiro
                pointer.execute("ROLLBACK")

                msg = '''
FROM.: create_table
AT...: ''' + str(datetime.utcnow()) + '''

Não foi possível criar tabela ''' + table_name + ''' devido ao erro: 
''' + str(error) + '''.

'''
   
                log_handler(msg,default_log)
    
    return create_table_job_status

                    
def read_table(table_name, **kwargs):
    
    """Busca no banco a tabela [table_name], retornando uma lista, tal que cada elemento representa
    uma linha (registro) da tabela, ou uma lista vazia, caso a tabela não seja encontrada. Pode retornar
    a tabela inteira, ou parte dos valores se forem informados, corretamente, os argumentos opcionais
    **kwargs.
    
    
    Keyword arguments:
    =================
    
    table_name  -- Nome da tabela a ser buscada
    
    **kwargs:
    =========
    
    mute -- Suprime o log se 'yes'
    
    field_key -- Chave (nome da coluna) do campo tomado como referência de ordenamento
    
    sort_type      -- Modo como os registros são buscados:
        
    *sort_type options*
     -----------------
        - DESC ==> Do maior para o menor
        - ASC  ==> Do menor para o maior
        
    limit     -- Número de registros que devem ser retornados
    """
    
    field_key = kwargs.get('field_key')
    
    sort_type = kwargs.get('sort_type')
    
    limit = kwargs.get('limit')

    mute = kwargs.get('mute')
    
    connection = '' #Inicialmente definida como string
    
    records = [] #Inicialmente definido como uma lista vazia.
    
    default_log = 'print'

    if(mute):
        
        if(mute == 'yes'): default_log = 'mute'
    
    #Tenta conectar ao banco
    try:

        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)

        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
        
        msg = '''
FROM.: read_table
AT...: ''' + str(datetime.utcnow()) + '''

Conectado ao banco com sucesso!

'''

        log_handler(msg,default_log)

    #Não conecta ao banco    
    except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI

        msg = '''
FROM.: read_table
AT...: ''' + str(datetime.utcnow()) + '''

Não foi possível conectar ao banco devido ao erro: 
''' + str(error) + '''

'''
        
        log_handler(msg,default_log)

    finally:

        if (type(connection) != str): #Conexão bem sucedida com o banco de dados
            
            #Tenta obter a tabela
            try:
                
                sql_select_query = 'SELECT * FROM ' + table_name
                
                if (field_key and sort_type and limit):
                    
                    sql_select_query = sql_select_query + ' ORDER BY ' + field_key + ' ' + sort_type + ' LIMIT ' + limit

                pointer.execute(sql_select_query)

                records= pointer.fetchall()
                
                msg = '''
FROM.: read_table
AT...: ''' + str(datetime.utcnow()) + '''

Tabela ''' + table_name +''' obtida com sucesso! 

'''
                
                log_handler(msg,default_log)

            #Não encontra tabela
            except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI
                
                #Desfaz ação do ponteiro
                pointer.execute("ROLLBACK")

                msg = '''
FROM.: read_table
AT...: ''' + str(datetime.utcnow()) + '''

Não foi possível obter tabela ''' + table_name +''' devido ao erro: 
''' + str(error) + '''

'''
                
                log_handler(msg,default_log)
            
            pointer.close()
            connection.close()
    
    return records


def update_table(table_name, pk_field, pk_value, field_to_update, new_field_value, **kwargs):
  
    """Dada uma certa tabela, atualiza uma entrada específica da mesma.
    
    
    Keyword arguments:
    =================
    
    table_name      -- Nome da tabela a ser buscada
    pk_field        -- Chave (nome da coluna) atribuída como chave primária; ``pk``, se chave default.
    pk_value        -- Valor da chave primária do registro
    field_to_update -- Chave (nome da coluna) do campo a ser regravado
    new_field_value -- Novo valor a ser gravado no campo
    
    **kwargs:
    =========
    
    mute -- Suprime o log se 'yes'
    """
    
    mute = kwargs.get('mute')

    connection = ''
    
    update_table_job_satus = 'fail'
    
    default_log = 'print'
    
    if(mute):
        
        if(mute == 'yes'): default_log = 'mute'
    
    #Tenta conectar ao banco
    try:

        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)

        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
    
    #Não conecta ao banco
    except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI
        
        msg = '''
FROM.: update_table
AT...: ''' + str(datetime.utcnow()) + '''

Não foi possível conectar ao banco devido ao erro: 
''' + str(error) + '''

'''
        log_handler(msg,default_log)
        
    finally:

        if (type(connection) != str): #Conexão bem sucedida com o banco de dados
            
            #Tenta atualizar entrada na tabela
            try:

                sql_update_query = 'Update ' + table_name + ' set ' + field_to_update + ' = %s where ' + pk_field + '= %s'
                
                pointer.execute(sql_update_query, (new_field_value, pk_value))

                connection.commit()

                update_table_job_satus = 'done'

            #Não consegue atualizar entrada
            except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI

                #Desfaz a ação do ponteiro
                pointer.execute("ROLLBACK")

                msg = '''
FROM.: update_table
AT...: ''' + str(datetime.utcnow()) + '''

Tabela ''' + table_name + ''': Não foi possível atualizar a entrada ''' + field_to_update + ''' 
do registro ''' + pk_value + ''' para ''' + new_field_value + ''' devido ao erro: 
''' + str(error) + '''

'''
   
                log_handler(msg,default_log)
    
            pointer.close()
            connection.close()

    return update_table_job_satus


def save_in_table(table_name, keys, data, **kwargs):
    
    """Salva na tabela <table_name>, nas respectivas colunas <keys[i]>, os dados correspondentes, <data>.
    
    
    Keyword arguments:
    =================
    
    table_name  -- Nome da tabela na qual os dados devem ser gravados
    keys        -- Chaves (nomes das colunas)
    data_list   -- Lista de dados a serem gravados


    **kwargs:
    ========

    mute -- Suprime o log se 'yes'
    """

    mute = kwargs.get('mute')

    connection = ''
    
    save_in_table_job_satus = 'fail'
    
    default_log = 'print'
    
    if(mute):
        
        if(mute == 'yes'): default_log = 'mute'
    
    #Tenta conectar ao banco
    try:

        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)

        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
    
    #Não conecta ao banco
    except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI
        
        msg = '''
FROM.: save_in_table
AT...: ''' + str(datetime.utcnow()) + '''

Não foi possível conectar ao banco devido ao erro: 
''' + str(error) + '''.

'''
        
        log_handler(msg,default_log)
        
    finally:

        if (type(connection) != str): #Conexão bem sucedida com o banco de dados
            

            #Tenta salvar os dados na tabela
            try:
                
                sql_save_query = sql_command(table_name, keys, 'save')
                
                for i in range(len(data)):
                    
                    pointer.execute(sql_save_query, data[i])
                
                connection.commit()
                
                save_in_table_job_satus = 'done'

            except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI

                #Desfaz ação do ponteiro
                pointer.execute("ROLLBACK")

                msg = '''
FROM.: save_in_table
AT...: ''' + str(datetime.utcnow()) + '''

Não foi possível salvar dados na tabela: ''' + table_name + ''' devido ao erro: 
''' + str(error) + '''.

'''

                log_handler(msg,default_log)
    
    
    return save_in_table_job_satus

In [3]:
#binance functions

#Import
import requests
import json
import time

#ESCREVER DOCSTRING
def binance_klines_request_handler(pair, interval, start_time, max_attempts):
    
    num_requests = 0
    
    klines = []
    
    url = '''https://api.binance.com/api/v1/klines?symbol=''' + pair + '''&interval=''' + interval + '''&startTime=''' + start_time
    
    default_log = 'print'
    
    for i in range(max_attempts):
        
        #Tenta conexão com a Binance
        num_requests+= 1
        try:

            response = requests.get(url)
            
            response.raise_for_status()
                        
        #Request falhou
        except (Exception, requests.exceptions.RequestException) as error: #TRATAR EXCEÇÃO AQUI
            
            msg = '''
FROM.: binance_klines_request_handler
AT...: ''' + str(datetime.utcnow()) + '''

Par ''' + pair + ''': Comunicação com a Binance falhou no ''' + str(i+1) + '''\u00bA request, devido ao erro:

''' + str(error) + '''

'''
            
            log_handler(msg,default_log)
            
            time.sleep(5)
            
        finally:
            
            if (int(response.status_code) == 200):
                
                klines = response.json()
                
                break

    return klines, num_requests

In [4]:
#Other functions

#Import

import sys
import os
from datetime import datetime
from modules import notification

def log_handler(msg,destination):
    
    """Envia uma mensagem de log para o destino escolhido pelo usuário.
    
    
    Keyword arguments:
    =================
    
    msg         -- Mensagem de log
    destination -- Caminho padrão para saída da mensagem de log
    
    *destination options*
     -------------------
        - mute   ==> Ignora a mensagem
        - print  ==> Printa na tela
        - logm   ==> Envia para o arquivo padrão de monitoramento
        - logw   ==> Envia para o arquivo padrão de alertas
        - tlgrm  ==> Envia para o grupo de monitoramento do telegram
        - tlgrw  ==> Envia para o grupo de alertas do telegram    
    """
    
    if (destination == 'mute'):
        
        pass
    
    elif (destination == 'print'):
        
        print(msg)
    
    elif (destination == 'logm'):
        
        path_to_logfile = os.getcwd().split(sep="BinanceDataStorageDaemon/")[0] + 'logs/binance_data_monitoring.log'
        sys.stdout = open(path_to_logfile, "a")
        print(msg)
        
    elif (destination == 'logw'):
        
        path_to_logfile = os.getcwd().split(sep="BinanceDataStorageDaemon/")[0] + 'logs/binance_data_warning.log'
        sys.stdout = open(path_to_logfile, "a")
        print(msg)
        
    elif (destination == 'tlgrm'):
        
        notification.Telegrambot(msg).monitoring()
        
    elif (destination == 'tlgrw'):
        
        notification.Telegrambot(msg).warning()


def sql_command(table_name, keys, action, **kwargs):
    
    """Dado um nome de tabela <table_name> e o dicionário <keys> (em que a chave será o nome da coluna e 
    o valor definirá o tipo de dado gravado - vide documentação do PostgreSQL), retorna o comando SQL ade-
    quado para criação da tabela ou salvamento de dados em uma tabela.
    
    
    Keyword arguments:
    =================
    
    table_name  -- Nome da tabela a ser buscada
    keys        -- Dicionário com as chaves e seus respectivos tipos
    action      -- O tipo de ação desejada com o comando SQL, criar ou salvar.

    *action options*
     ------------
        - create ==> SQL para criar uma tabela
        - save   ==> SQL para salvar em uma tabela
    
    **kwargs:
    =========
    
    pk -- Chave primária
    """
    
    pk = kwargs.get('pk')
    
    key_list = list(keys.keys())
    type_list = list(keys.values())
    
    sql = ''
    
    if (action == 'create'):
        
        sql = 'CREATE TABLE ' + table_name + ' ('
        
        for i in range (len(keys)-1):
            
            entry = key_list[i] + ' ' + type_list[i]
            
            if (pk):
                
                if (key_list[i] == pk):
                    
                    entry = key_list[i] + ' ' + type_list[i] + ' PRIMARY KEY'
                        
            sql = sql + entry + ', '

        sql = sql + key_list[len(keys)-1] + ' ' + type_list[len(keys)-1] + ')'
        
    if (action == 'save'):
        
        format_code = ''
        
        sql = 'INSERT INTO ' + table_name + ' ('
        
        for i in range (len(keys)-1):
            
            sql = sql + key_list[i] + ' ' + ', '
            
            format_code = format_code + '%s, '

        sql = sql + key_list[len(keys)-1] + ' ' + ') VALUES (' + format_code + '%s)'
                        
    return sql

#ESCREVER DOCSTRING
def binance_klines_to_postgres_klines(in_dt):
    
    out_dt = []
    
    for i in range (len(in_dt)):
        
        data = (datetime.fromtimestamp(in_dt[i][0]/1000), in_dt[i][1], in_dt[i][2], in_dt[i][3], 
                     in_dt[i][4], in_dt[i][5], in_dt[i][7], in_dt[i][8], in_dt[i][9], in_dt[i][10])
    
        out_dt.append(data)
    
    return out_dt

In [5]:
table_name = 'binance_klines_linkusdt_1m'


last_saved_candle = read_table(table_name, field_key = 'open_time',
                                    sort_type = 'DESC', limit = '1')#, mute = 'yes')

#open_time = last_saved_candle[0]['open_time']
#if (delta != 0): open_time = int(last_saved_candle[0]['open_time']) - delta

#start_time = str(int(datetime.timestamp(open_time)*1000) + 30000)


FROM.: read_table
AT...: 2019-06-22 01:05:47.466432

Conectado ao banco com sucesso!



FROM.: read_table
AT...: 2019-06-22 01:05:47.468293

Tabela binance_klines_linkusdt_1m obtida com sucesso! 




In [6]:
(last_saved_candle[0]['open_time'])

datetime.datetime(2019, 6, 21, 0, 57)

In [9]:
datetime.timestamp(last_saved_candle[0]['open_time'])

1561088340.0

In [51]:
binance_hour = datetime.fromtimestamp(int((requests.get('https://api.binance.com/api/v1/time').json()['serverTime'])/1000))

print('binance_hour = ', binance_hour)

utc_hour = datetime.utcnow()

delta = utc_hour - binance_hour

#delta = delta_hour*3600

print(delta.total_seconds())

#delta.total_


round(delta.total_seconds()/3600)

# A vulnerabilida está em subtrair só as horas. Subtrai-se os timestamps, caso a subtração seja maior que 1h, converte

binance_hour =  2019-06-21 22:33:59
10756.668023


3

In [47]:
binance_hour = datetime.fromtimestamp(int((requests.get('https://api.binance.com/api/v1/time').json()['serverTime'])/1000)).hour

utc_hour = datetime.utcnow().hour

print(utc_hour)
print(binance_hour)

delta_hour = utc_hour - binance_hour

delta = delta_hour*3600

print(delta.total_seconds())

1
22


AttributeError: 'int' object has no attribute 'total_seconds'

In [46]:
delta.total_seconds()

10756.535444

In [14]:
binance_hour = datetime.fromtimestamp(int((requests.get('https://api.binance.com/api/v1/time').json()['serverTime'])/1000))

print('binance_hour = ', binance_hour)

binance_hour =  2019-06-21 22:10:04


In [26]:
utc_hour = datetime.utcnow()
print(utc_hour)

2019-06-22 01:18:55.151660


In [28]:
datetime(2019, 6, 21, 0, 57)

datetime.datetime(2019, 6, 21, 0, 57)

In [30]:
timestamp_original = datetime.timestamp(datetime(2019, 6, 21, 22, 17))

print(timestamp_original)

1561166220.0


In [31]:
datetime.fromtimestamp(timestamp_original + 10800)

datetime.datetime(2019, 6, 22, 1, 17)

In [ ]:
int(datetime.timestamp(last_saved_candle[0]['open_time']))

In [13]:
binance_hour = datetime.fromtimestamp(int((requests.get('https://api.binance.com/api/v1/time').json()['serverTime'])/1000)).hour

utc_hour = datetime.utcnow().hour

delta_hour = utc_hour - binance_hour

delta = delta_hour*3600

In [14]:
datetime.timestamp(last_saved_candle[0]['open_time']) - delta

1561157520.0

In [15]:
datetime.fromtimestamp(1561157520.0)

datetime.datetime(2019, 6, 21, 19, 52)

# Zona de testes

## 1 - Funções banco de dados

### 1.1 - create_table

In [273]:
#=======================================================================
#Função auxiliar para gerar uma string randômica
import random
import string

def randomString(stringLength=10):
    
    """Generate a random string of fixed length """
    
    letters = string.ascii_lowercase
    
    return ''.join(random.choice(letters) for i in range(stringLength))
#=======================================================================

recent_created_table_name = 'test_table_' + randomString(10)

recent_created_table_name = 'binance_klines_ethbtc_1m'

keys = {'open_time': 'timestamp', 'open': 'numeric', 'high': 'numeric', 'low': 'numeric', 'close': 'numeric',
        'volume': 'numeric', 'quote_asset_volume': 'numeric', 'number_of_trades': 'integer',
       'taker_buy_base_asset_volume': 'numeric', 'taker_buy_quote_asset_volume': 'numeric'}

create_table_job_satus = create_table(recent_created_table_name,keys,pk='open_time')

print(create_table_job_satus)

done


### 1.2 - read_table

In [270]:
#Coletando todas as amostras

table_name = 'binance_pairs'

records = read_table(table_name, mute='yes')

if (len(records) > 0):
    
    for record in records:
        
        print (record)

['BTCUSDT', 'OFF', 'building', None, None]
['XRPUSDT', 'OFF', 'absent', None, None]
['ADAUSDT', 'OFF', 'building', None, None]
['ETHUSDT', 'OFF', 'full', None, None]
['BNBUSDT', 'ON', 'full', None, None]


In [271]:
import os

In [272]:
os.getcwd()

'/home/marcus/00-projetos/OpenBot/docker-cluster/BinanceDataStorageDaemon'

In [151]:
#Coletando uma parte das amostras

table_name = 'binance_klines_ethbtc_1m'

records = read_table(table_name, field_key = 'open_time', sort_type = 'DESC', limit = '1')

if (len(records) > 0):
    
    #print('''open_time | open | high | low | close | volume | quote_asset_volume | number_of_trades | 
          #taker_buy_base_asset_volume | taker_buy_quote_asset_volume | 
          #''')
    
    for record in records:
        
       # print (record['open_time'], record['open'], record['high'], record['low'], record['close'], 
               #record['volume'], record['quote_asset_volume'], record['number_of_trades'], record['taker_buy_base_asset_volume'], record['taker_buy_quote_asset_volume'])
        print (record[0])
        print(' ')


FROM: read_table
2019-06-12 17:51:47.544398

Conectado ao banco com sucesso!



FROM: read_table
2019-06-12 17:51:47.545101

Não foi possível obter tabela binance_klines_ethbtc_1m devido ao erro: 
relation "binance_klines_ethbtc_1m" does not exist
LINE 1: SELECT * FROM binance_klines_ethbtc_1m ORDER BY open_time DE...
                      ^





### 1.3 - update_table

In [9]:
#Testando para mudança da chave 'status', da tabela 'binance_pairs' Modifique À vontade.

#=======================================================================
#Função auxiliar para gerar uma string randômica
import random
import string

def randomString(stringLength=10):
    
    """Generate a random string of fixed length """
    
    letters = string.ascii_lowercase
    
    return ''.join(random.choice(letters) for i in range(stringLength))
#=======================================================================


table_name = 'binance_pairs'

pk_field = 'name'

pk_value = 'BTCUSDT'

field_to_update = 'status'

new_field_value = randomString(8) #Máximo 8 caracteres

records = read_table(table_name, mute='yes')

for record in records:

    if (record[pk_field] == pk_value):

        old_record = record
        
        old_field_value = record[field_to_update]

update_table_job_status = update_table(table_name, pk_field, pk_value, field_to_update, new_field_value)

print(update_table_job_status + '''
''')

if (update_table_job_status == 'done'):
    
    records = read_table(table_name, mute='yes')
    
    for record in records:
        
        if (record[pk_field] == pk_value):
            
            new_record = record
            
    #Desfazendo
    update_table_job_status = update_table(table_name, pk_field, pk_value, field_to_update, old_field_value)
    
print('''Entrada anterior   - ''' + str(old_record) + '''
Entrada atualizada - ''' + str(new_record) + '''
''')

done

Entrada anterior   - ['BTCUSDT', 'OFF', 'absent', None, None]
Entrada atualizada - ['BTCUSDT', 'OFF', 'aecxewcd', None, None]



### 1.4 - save_in_table


In [10]:
#=======================================================================
#Função auxiliar para gerar uma string randômica
import random
import string

def randomString(stringLength=10):
    
    """Generate a random string of fixed length """
    
    letters = string.ascii_lowercase
    
    return ''.join(random.choice(letters) for i in range(stringLength))
#=======================================================================

recent_created_table_name = 'test_table_' + randomString(10)

keys = {'open_time': 'timestamp', 'open': 'numeric', 'high': 'numeric', 'low': 'numeric', 'close': 'numeric',
        'volume': 'numeric', 'quote_asset_volume': 'numeric', 'number_of_trades': 'integer',
       'taker_buy_base_asset_volume': 'numeric', 'taker_buy_quote_asset_volume': 'numeric'}

create_table_job_satus = create_table(recent_created_table_name,keys,pk='open_time',mute='yes')

if (create_table_job_satus == 'done'):
    
    print ('Nova tabela criada com o nome: ',recent_created_table_name)

    import requests
    import json

    url = 'https://api.binance.com/api/v1/klines?symbol=BTCUSDT&interval=1m&startTime=1241893500000'

    response = requests.get(url)

    klines = response.json()

    #for kline in klines:

    klines = binance_klines_to_postgres_klines(klines)

    save_in_table_job_status = save_in_table(recent_created_table_name, keys, klines)

    print ('save_in_table_job_status = ',save_in_table_job_status)

Nova tabela criada com o nome:  test_table_iwjbbzrkbu
save_in_table_job_status =  done


## 2 - Funções da binance

### 2.1 - binance_klines_request_handler

In [ ]:
pair = 'ETHUSDT'
interval = '1m'
start_time = '1241893500000'
max_attempts = 10

klines, num_requests = binance_klines_request_handler(pair, interval, start_time, max_attempts)

print('O total de requests foi: ', num_requests)
print(' ')


if (len(klines) > 0):
    
    
    for i in range (5):
        
        print(klines[i] , '''
        ''')

## 3 - data_completer

In [ ]:
keys = {'open_time': 'timestamp', 'open': 'numeric', 'high': 'numeric', 'low': 'numeric', 'close': 'numeric',
        'volume': 'numeric', 'quote_asset_volume': 'numeric', 'number_of_trades': 'integer',
       'taker_buy_base_asset_volume': 'numeric', 'taker_buy_quote_asset_volume': 'numeric'}

pair = 'BTCUSDT'

interval = '1m'

start_time = '1241893500000'

max_requests = 1000

table_name = 'binance_klines_' + pair + '_' + interval

#print(table_name)

create_table_job_satus = create_table(table_name,keys,pk='open_time')

data_completer(pair, interval, start_time, max_requests)